# [이론4] 문서 유사도 & 언어 모델(Document Similarity & Language Model)

---

## 학습 목표
- 문서 표현의 다른 방법인 vector space model에 대해 살펴봅시다.
- 문서 유사도를 구하는 방법에 대해 살펴봅시다.
- 언어 모델에 대해 살펴봅시다.

---

## 목차

### [이론4] 문서 유사도 & 언어 모델(Document Similarity & Language Model)
1. 소개
2. 문서 유사도
3. 언어 모델

---

##  [이론4] 문서 유사도 & 언어 모델(Document Similarity & Language Model)

### 1. 소개

이번 시간에는 문서의 유사도 그리고 언어 모델(Language Model)에 대해 살펴보겠습니다.

이번 시간 내용은 딥러닝 이전에 쓰여졌던 방법론들을 중점으로 살펴볼 예정입니다.

하지만 해당 내용은 자연어처리를 배운 분이라면 알아야 하는 개념이기에 잘 살펴봐주시길 바랍니다.

텍스트 표현을 배울 때 우리는 단어 간의 유사도를 word2vec과 fastText로 확인하였습니다.

그렇다면 단어의 집합인 문서의 경우 유사도 측정이 필요할까요?

문서 유사도를 가장 많이 사용하고 연구하는 분야는 바로 검색 엔진입니다.

검색 엔진은 사용자의 검색어를 입력으로 받아 그에 적합한 문서를 찾아줍니다.

그렇기에 검색 엔진이 크롤링한 문서에 대한 이해는 검색 엔진이 가져야 할 필수적인 요소이죠.

또한 사용자의 검색어 역시 하나의 문서로 볼 수 있습니다.

즉, 검색어 문서와 크롤링한 문서의 유사도를 계산하여 가장 적합한 문서를 결과로 보여주는 것이 검색 엔진이라고 볼 수 있습니다.

이번 시간에는 이러한 문서 유사도를 구하는 방법에 대해 살펴볼 것입니다.

더하여 텍스트에 대한 좀 더 깊은 이해를 담은 모델인 언어 모델에 대해서도 살펴볼 것입니다.

### 2. 문서 유사도

문서 유사도를 구하기 위해서는 먼저 우리는 문서 표현에 대해서 살펴봐야합니다.

문서 표현은 이전 시간에 배운 것처럼 RNN을 통해 만들 수 있습니다.

그리고 다른 방법도 있는데 그 중 이번 시간에 소개할 것이 바로 bag of words 입니다.

이전에 fastText를 배웠을 때 n-gram에 대해 살펴보았습니다.

n-gram이란 시계열(sequence) 형태의 것을 길이에 따라 쪼개는 방법으로 이 때 쪼개어진 것의 개수가 최대가 되도록 하나씩 움직이며 쪼개는 것을 뜻합니다.

<img src="img/fasttext_lose.jpg" width="50%" height="50%" title="fasttext_lose" alt="fasttext_lose"></img>

이 때 n이 1인 경우를 생각해봅시다.

위의 그림 속 예시에서 n이 1인 경우는 아래와 같은 단어 파편(subword)이 나올 수 있습니다.

> `<`, `l`, `o`, `s`, `e`, `>`

이렇게 하나의 길이만을 가지도록 표현한 것이 unigram이라 불립니다. (1-gram이 아닙니다)

이렇게 unigram 방법을 이용하여 문서를 표현해보겠습니다.

단어를 하나의 단위로 하여 unigram을 수행하면 문서 속 단어들을 추출할 수 있습니다.

그 후 해당 문서의 단어들의 개수를 세어 그 문서의 표현을 만들 수 있습니다.

<img src="img/bagofwords.jpg" width="50%" height="50%" title="bagofwords" alt="bagofwords"></img>

출처: https://www.programmersought.com/article/4304366575/

위의 그림이 그 한 예시입니다.

왼쪽의 문서에서 단어들을 쪼개어 이를 bag에 담습니다.

그리고 그 bag에서 단어들을 꺼내어 정렬한 후 각 단어의 개수를 세면 오른쪽의 형태, 즉 문서 표현이 되는 것입니다.

이러한 문서 표현을 통해 문서 유사도를 살펴보겠습니다.

#### Jaccard Similarity

앞서 살펴본 bag of words 문서 표현을 사용하여 문서 유사도를 구하는 방법 중 가장 간단한 방법으로 Jaccard similarity가 있습니다.

Jaccard similarity의 아이디어는 매우 간단합니다.

두 집합 $A, B$에서 교집합 $A \cap B$와 합집합 $A \cup B$를 구하여 그 둘의 크기를 비교하는 것입니다.

이를 문서 유사도에 적용하면 두 문서 $D_1, D_2$가 있을 때 각 문서를 bag of words 문서 표현으로 만듭니다.

해당 문서 표현은 단어들의 집합으로 볼 수 있습니다.

따라서 두 문서의 단어들의 교집합과 합집합으로 구할 수 있습니다.

이를 정리하면 아래와 같습니다.

Jaccard similarity $D_1, D_2$ = $\frac{|D_1 \cap D_2|}{|D_1 \cup D_2|}$

실제 코드와 함께 살펴보겠습니다.

In [27]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

document_1 = 'South Korea is a country in East Asia, constituting the southern part of the Korean Peninsula, and sharing a land border with North Korea. 25 million people, around half of the countrys population of more than 51 million people, live in the Seoul Capital Area, the fifth-largest metropolitan area in the world.'
document_2 = 'North Korea is a country in East Asia constituting the northern part of the Korean Peninsula. The country is bordered to the north by China and by Russia along the Amnok and Tumen rivers, and to the south by South Korea, with the heavily fortified Korean Demilitarized Zone (DMZ) separating the two.'

word_tokens_document_1 = word_tokenize(document_1)
word_tokens_document_2 = word_tokenize(document_2)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
union_tokens = set(word_tokens_document_1 + word_tokens_document_2)
intersection_tokens = set(word_tokens_document_1).intersection(set(word_tokens_document_2))
jaccard_similarity = len(intersection_tokens) / len(union_tokens)
print(jaccard_similarity)

0.30158730158730157


위의 예시에서 두 문서는 각각 남한과 북한을 소개한 위키피디아 문서에서 일부 가져왔습니다.

- https://en.wikipedia.org/wiki/South_Korea
- https://en.wikipedia.org/wiki/North_Korea

두 문서를 nltk로 단어 토큰을 만든 후 이를 통해 Jaccard similarity를 구했습니다.

#### Vector Space Model



문서에서 중요한 단어는 어떤 단어일까요?

아무래도 많이 나타나는 단어가 그 문서에서 중요한 단어일 것입니다.

중요한 단어이니만큼 강조하면서 여러 번 얘기될 것이기 때문입니다.

앞에서 살펴보았던 Jaccard similarity는 여러 문제가 있습니다.

그 중 가장 큰 문제는 문서 내 단어의 개수를 신경쓰지 않았다는 것입니다.

단어가 여러 번 사용되었음에도 이를 무시하고 단어의 존재여부만을 가지고 문서 유사도를 계산한 것입니다.

그럼 bag of words 문서 표현에서 문서를 벡터로 만들어 보겠습니다.

벡터에는 차원이 존재합니다.

해당 문서 표현에서 하나의 차원은 하나의 단어를 뜻합니다.

한 문서의 벡터 표현은 각 단어가 몇 번 나타났는지를 가지고 표현할 수 있습니다.

예시를 코드와 함께 보겠습니다.

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document. This is very important.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
    'This is Sparta'
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print()
print(X.toarray())

['and', 'document', 'first', 'important', 'is', 'one', 'second', 'sparta', 'the', 'third', 'this', 'very']

[[0 1 1 1 2 0 0 0 1 0 2 1]
 [0 2 0 0 1 0 1 0 1 0 1 0]
 [1 0 0 0 1 1 0 0 1 1 1 0]
 [0 1 1 0 1 0 0 0 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 1 0]]


위의 코드에서 corpus 변수에 총 다섯 개의 텍스트 문서가 있습니다.

그리고 이를 scikit-learn의 CountVectorizer를 이용하여 문서를 벡터로 만들었습니다.

마지막에 출력된 행렬의 행은 각 문서를 뜻하고 열은 각 단어를 뜻합니다.

첫 번째 열은 'and', 두 번째 열은 'document' 단어를 뜻하는 것을 알 수 있습니다.

그리고 행렬 속 값은 해당 단어가 몇 번 나타났는지를 보여줍니다.

이렇게 문서를 벡터로 바꾸어서 표현한 것을 Vector Space Model이라고 부릅니다.

물론 지금 보여드린 것은 그러한 vector space model의 가장 간단한 것으로 이보다 발전된 것은 다양하게 많습니다.

그 중 하나인 Term Frequency - Inverse Document Frequency를 이용한 방법을 살펴보겠습니다.

#### Term Frequency - Inverse Document Frequency

다시 물어보겠습니다.

문서에서 중요한 단어는 어떤 단어일까요?

아무래도 많이 나타나는 단어가 그 문서에서 중요한 단어일 것입니다.

중요한 단어이니만큼 강조하면서 여러 번 얘기될 것이기 때문입니다.

하지만 많이 나타났다고 해서 그 문서에 중요한 단어라고 할 수 있을까요?

예를 들어 앞서 살펴본 corpus 변수 내 문서를 보겠습니다.

In [30]:
corpus = [
    'This is the first document. This is very important.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
    'This is Sparta'
]
print("\n".join(corpus))

This is the first document. This is very important.
This document is the second document.
And this is the third one.
Is this the first document?
This is Sparta


위 첫 번째 문서를 보면 `this`라는 단어가 다른 단어들에 비해 많이 나타납니다.

과연 이 단어는 첫 번째 문서에서 중요한 단어일까요?

그렇다고 하기에는 조금 문제가 있습니다.

왜냐하면 해당 단어는 첫 번째 문서뿐만 아니라 모든 문서에서 나타나기 때문입니다.

그렇기에 그 단어가 첫 번재 문서에서 중요한 단어라고 하기에는 어려움이 있습니다.

다시 말하자면 `this`가 첫 번째 문서를 대표하는 단어라고 하기에는 한계가 있습니다.

이와 같은 아이디어에서 생각한 것이 document frequency 입니다.

하나의 단어가 여러 문서에서 얼마나 나타나는지를 계산하는 것입니다.

예를 들어 위의 `this`의 경우 총 다섯 문서에서 나타났으니 document frequency는 `5`가 됩니다.

`document`의 경우도 첫 번째, 두 번째, 네 번째 문서에서 나왔으니 document frequency는 `3`이 됩니다.

이 때 핵심은 한 문서에 몇 번 나타났느냐는 중요하지 않고 해당 단어가 문서에 존재하는지만을 본다는 점입니다.

이를 정리한 것이 Term Frequency - Inverse Document Frequency 입니다.

하나의 단어가 한 문서에서 얼마나 중요한지를 표현하는 용도로 쓰여지는 것입니다.

한 단어가 해당 문서에 몇 번 나타나는지(term frequency)를 측정합니다.

그리고 그 단어가 전체 문서에서 몇 번 나타나는지(document frequency)를 측정합니다.

그 후 document frequency에 역수를 취한 후 이를 term frequency와 곱한 것이 바로 Term Frequency - Inverse Document Frequency 줄여서 tf-idf 라고 불리는 것입니다.

이렇게 각 단어의 중요도를 tf-idf로 구한 후 앞서 살펴본 vector space model에 가중치로 곱하여 문서를 표현할 수 있습니다.

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
corpus = [
    'This is the first document. This is very important.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
    'This is Sparta'
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print()
print(X.toarray())

['and', 'document', 'first', 'important', 'is', 'one', 'second', 'sparta', 'the', 'third', 'this', 'very']

[[0.         0.29275244 0.35267539 0.43713206 0.41659154 0.
  0.         0.         0.24627258 0.         0.41659154 0.43713206]
 [0.         0.70933829 0.         0.         0.25235002 0.
  0.52958485 0.         0.29835887 0.         0.25235002 0.        ]
 [0.51492278 0.         0.         0.         0.24536346 0.51492278
  0.         0.         0.29009851 0.51492278 0.24536346 0.        ]
 [0.         0.48961805 0.58983706 0.         0.34836727 0.
  0.         0.         0.41188214 0.         0.34836727 0.        ]
 [0.         0.         0.         0.         0.39515588 0.
  0.         0.829279   0.         0.         0.39515588 0.        ]]


위의 코드는 같은 `corpus`에 대해서 tf-idf를 고려하여 만든 vector space model입니다.

이전과 달리 각 단어의 가중치가 실수로 표현된 것을 알 수 있습니다. 이 각각의 값이 바로 한 단어가 해당 문서에서 가지는 tf-idf 점수입니다.

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(X[0], X[1]).flatten()
print("Similarity between first and second documents: {}".format(cos_sim))

Similarity between first and second documents: [0.49139188]


문서를 vector로 표현하였으니 이제 각 문서들의 유사도를 cosine similarity로 구할 수 있습니다.

위의 코드는 이를 구한 것으로 첫 번째 문서와 두 번째 문서의 유사도를 알 수 있습니다.

cosine similarity는 어떻게 계산하는 것일까요?

두 벡터 $a$, $b$가 있을 때 cosine similarity는 아래와 같이 계산할 수 있습니다.

$$cos(a, b) = \frac{a \cdot b}{||a|| ||b||} = \frac{\sum_i^n a_i b_i}{\sqrt{\sum_i^n a_i^2} \sqrt{\sum_i^n b_i^2} }$$

즉, 두 벡터의 내적을 각 벡터의 크기(norm)의 곱으로 나눈 것이 cosine similarity가 됩니다.

이렇게 두 벡터의 유사도 혹은 거리를 측정하는 방법으로 유명한 것은 Euclidean Distance라는 것이 있습니다.

우리가 수학 시간에 배웠던 두 점 사이의 거리를 계산하는 것이 바로 Euclidean Distance입니다.

$$E(a, b) = \sqrt{(a_1 - b_1)^2 + (a_2 - b_2)^2 + \cdots + (a_n - b_n)^2}$$

그럼 왜 vector space model에서는 Euclidean Distance가 아닌 Cosine similarity를 쓰는 것일까요?

두 방법의 가장 큰 차이는 살펴보는 것이 다르다는 점입니다.

Cosine similarity는 두 벡터의 각도만을 계산하지만 Euclidean Distance는 각도와 함께 각 벡터의 크기까지 고려합니다.

이 때 Euclidean Distance를 두 문서 벡터의 거리/유사도로 계산할 경우 한 벡터의 크기가 그 거리/유사도에 큰 영향을 미치는 문제가 있습니다.

예를 들어 한 문서에서 특정 단어가 많이 등장하였다면 그만큼 해당 단어 차원에 대해 큰 값을 가지게 됩니다.

이 때 Euclidean Distance는 그 단어가 얼마나 존재하느냐에 따라 다른 문서들과 유사도를 계산하게 될 것이고 그럼 다른 단어들은 무시되는 경향이 있습니다.

하지만 Cosine similarity는 두 벡터의 각도만을 보기에 이런 문제점을 해결할 수 있다는 장점이 있습니다.

### 3. 언어 모델

언어 모델에 대해 살펴보기 앞서 다음 두 문장에 대해 생각해봅시다.

- 나는 인공지능 자연어처리 공부가 좋다
- 나는 좋다 자연어처리 공부가 인공지능

위의 두 문장은 서로 같은 단어들을 쓰고 있습니다. 그래서 만약 

위의 두 문장 중 어떤 문장이 맞는 문장일까요?

한국어를 아시는 분은 첫 번째 문장이 올바른 문장이라는 것을 알 수 있습니다.

하지만 앞서 살펴본 bag of words 방법으로 접근한다면 저 두 문장은 같은 문서 표현을 가지게 될 것입니다.

왜냐하면 두 문장이 가지는 단어와 그 단어 개수가 동일하기 때문입니다.

이처럼 bag of words는 문서 내 단어들 간의 순서를 고려하지 않는다는 문제가 있습니다.

이를 극복하기 위해 여러 모델들이 제시되었으며 이러한 모델들을 언어 모델이라고 부릅니다.

즉, 문장이나 문서가 얼마나 올바른지에 대한 것을 학습하는 모델이 바로 언어 모델입니다.

좀 더 자세하게 언어 모델에 대해 설명하겠습니다.

언어 모델이란 어떤 문장 혹은 연속된 단어 조합이 주어졌을 때 그 단어가 생성될 확률을 계산합니다.

단어 $k$개를 가지는 텍스트 $W$가 있을 때 언어 모델은 아래 확률을 계산하는 것입니다.

$P(W) = P(w_1 w_2 \ldots w_k)$

예를 들어 좋은 언어 모델이 있을 때 아래 두 문장의 확률을 다음과 같이 나타낼 것입니다.

- P(나는 인공지능 자연어처리 공부가 좋다) = 0.7
- P(나는 좋다 자연어처리 공부가 인공지능) = 0.2

이를 통해 우리는 첫 번째 문장이 두 번째 문장보다 올바른 문장임을 알 수 있습니다.

이렇게 텍스트의 확률을 구할 수 있으면 어떤 것이 가능할까요?

앞의 단어가 주어졌을 때 그 뒤의 단어의 확률을 계산할 수 있습니다.

$P(w_k | w_1 w_2 \ldots, w_{k-1})$

이를 통해 우리는 주어진 텍스트에 다음 단어로 올 수 있는 것을 예측할 수 있습니다.

그 예시 중 하나가 바로 검색 엔진에서 검색어 확장입니다.

<img src="img/keywordexpansion.png" width="50%" height="50%" title="keywordexpansion" alt="keywordexpansion"></img>

위의 그림에서 알 수 있듯 `why people`이라는 텍스트가 주어졌을 때 그 뒤에 나올 적합한 단어를 보여주고 있습니다.

이러한 작업을 할 수 있게 하는 것이 바로 언어 모델의 힘입니다.

그럼 이러한 확률을 어떻게 계산하는 것일까요?

다시 한 번 n-gram에 대해 생각해봅시다.

n이 1이라면 unigram이라고 하였습니다.

그럼 텍스트 $W$에 대해 unigram을 사용한다면 아래와 같이 확률을 계산할 수 있습니다.

$P(W) = P(w_1) P(w_2) \ldots P(w_k)$

즉, 텍스트를 단어 하나씩 쪼개어 그 각각의 확률을 구한 후 한꺼번에 곱하는 방식입니다.

하지만 이 방법은 문제가 있음을 여러분은 이미 알고 있습니다.

바로 단어들 간의 순서가 고려되지 않았다는 것이지요.

그래서 이를 방지하고자 n을 조금 더 크게 보겠습니다.

n이 2라면 단어 두 개를 동시에 보는 것인데 그 중 왼쪽 것이 주어졌을 때 오른쪽 것의 확률을 구하는 방식으로 계산합니다.

(참고로 n이 2인 경우를 bigram이라고 부릅니다)

$P(W) = P(w_1) P(w_2|w_1) P(w_3 | w_2) \ldots P(w_k| w_{k-1})$

즉, 앞의 단어가 주어졌을 때 그 다음 단어가 나올 확률을 전부 다 구하여 이를 곱하는 방식입니다.

이런 bigram의 방식을 일반화하여 n-gram의 형태로 보도록 하겠습니다.

그럼 텍스트 $W$에 대해서 아래와 같이 계산할 수 있습니다.

$P(W)= P(w_1) P(w_2|w_1) P(w_3|w_1 w_2) \ldots P(w_k | w_{k-1} w_{k-2} w_{k-3} \ldots w_{k-n})$

즉, n의 길이만큼의 단어들이 주어졌을 때 그 다음 단어의 확률을 계산하여 이를 곱하는 방식입니다.

이러한 방식의 언어 모델을 n-gram model이라고 부릅니다.

이번 시간에는 문서 유사도와 언어 모델에 대해 살펴보았습니다.

자연어처리에 있어 중요한 문서의 표현 방법과 언어 모델에 대해 살펴보았습니다.

언어 모델의 종류로 n-gram model도 있지만 지난 시간에 살펴본 RNN 역시 언어 모델입니다.

왜 그런지에 대해 이번 시간에 나온 내용과 함께 생각해보시기를 추천드립니다.

다음 시간에는 BERT(Bidirectional Encoder Representations from Transformers)에 대해 살펴보도록 하겠습니다.

---

<span style="color:rgb(120, 120, 120)">본 학습 자료를 포함한 사이트 내 모든 자료의 저작권은 엘리스에 있으며 외부로의 무단 복제, 배포 및 전송을 불허합니다.

Copyright @ elice all rights reserved</span>